# Домашнее задание 4

В домашнем задании нужно обучить модель для ответов на вопросы. Будем использовать датасет SQUAD, включающий вопросы, контекст и ответы. 

Цель задания: поэкспериментировать с генерацией и проанализировать результаты. В качестве модели можно выбрать модель на основе декодера трансформера или модель с архитектурой Encoder-Decoder.

Баллы за ДЗ:



*   Предобработка и токенизатор - 2 балл
*   Загрузка и обучение модели - 3 балла
*   Инференс и эксперименты - 3 балла
*   Отчёт - 2 балла
* Бонус - 5 баллов


## Загузка датасета

In [1]:
!pip install transformers datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 96.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset, load_metric

In [3]:
ds = load_dataset('squad')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

## Предобработка и токенизатор

Далее нужно подготовить и токенизировать данные для обучения модели. 

В зависимости от выбора модели формат представления данных будет различаться.

* Для генеративной модели на основе декодера (GPT) нужно подготовить промпт в качестве условия для генерации. Например, данные для одного вопроса могут выглядеть так: '<context> Question: <question> Answer: <answer>'. С форматом промпта можно экспеиментировать.

* Для Encoder-Decoder модели (например, T5) нужно отдельно токенизировать входные данные (контекст и вопрос) и таргет (ответ). В начале инпута возможно нужно будет указать префикс задания для модели.



In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("t5-base")

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [7]:
def tokenize(q_a):
    model_inputs = tokenizer(q_a['question'], q_a['context'], max_length=1024, truncation=True)

    # Настройка токенизатора для целей
    answers = [answer['text'][0] for answer in q_a["answers"]]
    labels = tokenizer(answers, max_length = 128, truncation=True)

    # Добавление меток входных данных модели
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
ds_tokenized = ds.map(tokenize, batched=True)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

## Обучение модели

In [9]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [11]:
batch_size = 2
args = Seq2SeqTrainingArguments(
    "finetuned-t5",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=0.5,
    #fp16=True,
    predict_with_generate=True,
    push_to_hub=False,
)

In [12]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=28dd94d4a54e9c650b9c1351237a148e3da53e3159f710b36ed90217e6fc5dbe
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score


In [13]:
rouge_metric = load_metric("rouge")
meteor_metric = load_metric("meteor")

<ipython-input-13-aad19ce9d9d8>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [14]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    result.update(meteor_metric.compute(predictions=decoded_preds, references=decoded_labels))
    return {k: round(v, 4) for k, v in result.items()}

In [15]:
from transformers import Trainer
trainer = Trainer(
    model,
    args,
    train_dataset=ds_tokenized["train"],
    eval_dataset=ds_tokenized["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [16]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


OutOfMemoryError: ignored

## Инференс модели

Запустите модель на нескольких примерах с помощью пайплайна. Можно поэкспериментировать с декодированием, задать разные параметры генерации. Сравните результаты.

## Отчет

- Использована модель t5-base
- Я обучила модель, однако, на последнем шаге всегда OutOfMemoryError (я четыре раза меняла параметры, размер батчей, но все равно на последнем шаге вылетает)

(TODO: describe results)

## Бонус

В датасете SQUAD ответы на вопросы содержатся внутри контекста. Можно попробовать обучить модель на основе энкодера для предсказания начала и окончания ответа по токенам контекста:
![](https://media.springernature.com/lw685/springer-static/image/chp%3A10.1007%2F978-1-4842-6664-9_5/MediaObjects/498208_1_En_5_Fig2_HTML.jpg)